In [1]:
import pandas as pd
import geopandas as gp


In [8]:
num_cities = 16

# Colombia

In [4]:
shp_munic = gp.read_file('data/colombia/ADMINISTRATIVO/MGN_MPIO_POLITICO.shp')
col_munic = ['DPTO_CCDGO','MPIO_CCDGO', 'MPIO_CNMBR', 'DPTO_CNMBR', 'geometry']

# CLAS_CCDGO: 1-Municipal Headboard 2-populated center 3-scattered rural area 
# 1-Cabecera municipal, 2-Centro poblado y 3-Área rural dispersa.
shp_urb_censal = gp.read_file('data/colombia/MGN/MGN_URB_AREA_CENSAL.shp')
print(shp_urb_censal.shape)
shp_urb_censal=shp_urb_censal.merge(shp_munic[col_munic[:-1]])
shp_urb_censal.head()

(7627, 14)


,OBJECTID,DPTO_CCDGO,MPIO_CCDGO,CLAS_CCDGO,SETR_CCDGO,SECR_CCDGO,CPOB_CCDGO,CPOB_CNMBR,CPOB_CCNCT,CPOB_NAREA,CPOB_NALT,Shape_Leng,Shape_Area,geometry,MPIO_CNMBR,DPTO_CNMBR
0,1,05,05001,2,000,00,05001004,SANTA ELENA,05001200000004,0.338461,2540.0,0.034407,0.000028,POLYGON ((-75.49783904499998 6.207774365000034...,MEDELLÍN,ANTIOQUIA
1,2,05,05001,2,000,00,05001014,BARRO BLANCO,05001200000014,0.534495,2531.0,0.044447,0.000044,POLYGON ((-75.47379840599996 6.239590113000077...,MEDELLÍN,ANTIOQUIA
2,3,05,05001,2,000,00,05001015,EL CERRO,05001200000015,0.414935,2668.0,0.042879,0.000034,(POLYGON ((-75.49121943999995 6.20501584900006...,MEDELLÍN,ANTIOQUIA
3,4,05,05001,2,000,00,05001018,EL PLACER,05001200000018,0.965299,2598.0,0.056312,0.000079,"POLYGON ((-75.49276999399996 6.23050003000003,...",MEDELLÍN,ANTIOQUIA
4,5,05,05001,2,000,00,05001019,EL PLAN,05001200000019,3.252561,2700.0,0.093795,0.000266,POLYGON ((-75.50917742799999 6.195200505000059...,MEDELLÍN,ANTIOQUIA


In [5]:
population = pd.read_csv('data/colombia/population_2005.csv',sep='\t')
cols = ['TOTAL_2005','CABECERA_2005', 'RESTO_2005']
for col in cols:
    population[col] = population[col].apply(lambda x: float(x.replace(',', '')))
    
population['CABECERA_pcnt'] = population.CABECERA_2005/population.TOTAL_2005
population.sort_values('TOTAL_2005', ascending=False, inplace=True)
population['pop_order'] = pd.np.arange(population.shape[0])+1
population.DPMP = population.DPMP.apply(lambda x: '%05d' % x)

In [12]:
thres = 0.8
target_munic = population[population.CABECERA_pcnt>thres].head(num_cities)

In [16]:
target_cities = shp_urb_censal.merge(target_munic, left_on='MPIO_CCDGO', right_on='DPMP')
target_cities = target_cities[target_cities.CLAS_CCDGO=='1'].copy()

cols = ['OBJECTID', 'DPTO_CCDGO', 'MPIO_CCDGO', 'CLAS_CCDGO', 'CPOB_CCDGO', 'MPIO_CNMBR', 'MPIO', 'CPOB_CNMBR', 'CPOB_NAREA', 'TOTAL_2005',
       'CABECERA_2005', 'RESTO_2005', 'CABECERA_pcnt', 'pop_order', 'geometry']

target_cities = target_cities[cols]

In [27]:
target_cities.to_file('data/cities_col_16.geojson', driver="GeoJSON")

# Mex

In [29]:
from src.utils import fix_spanish_encoding

In [28]:
# http://www.inegi.org.mx/geo/contenidos/geoestadistica/m_g_0.aspx
shp_urban = gp.read_file('data/mexico/mglu2014v6_2/mglu2015v6_2.shp')
shp_state = gp.read_file('data/mexico/mge2014v6_2/mge2015v6_2.shp')
shp_munic = gp.read_file('data/mexico/mgm2014v6_2/mgm2015v6_2.shp')

In [30]:
shp_urban.NOM_LOC = shp_urban.NOM_LOC.apply(fix_spanish_encoding)

In [35]:
shp_all = shp_urban.merge(shp_state.drop('geometry', axis=1)).merge(shp_munic.drop('geometry', axis=1))
idx = [742, 3396, 454, 431, 2095, 3515, 1186, 48, 2280, 1943, 3815, 2373, 71, 1771, 3709]
shp_all.loc[idx]

,CVE_ENT,CVE_MUN,CVE_LOC,NOM_LOC,geometry,NOM_ENT,NOM_MUN,concat
742,12,001,0001,Acapulco de Juárez,"POLYGON ((-99.9395455851606 16.88883286234676,...",Guerrero,Acapulco de Juárez,12001
3396,23,005,0001,Cancún,POLYGON ((-86.83937210783648 21.21085723087657...,Quintana Roo,Benito Juárez,23005
454,08,037,0001,Juárez,(POLYGON ((-106.4025488769291 31.5450063380720...,Chihuahua,Juárez,08037
431,08,019,0001,Chihuahua,(POLYGON ((-106.1628344308643 28.5909839381582...,Chihuahua,Chihuahua,08019
2095,17,007,0001,Cuernavaca,POLYGON ((-99.25852772267621 19.00069855369537...,Morelos,Cuernavaca,17007
3515,25,006,0001,Culiacán Rosales,(POLYGON ((-107.420703891971 24.74156052279994...,Sinaloa,Culiacán,25006
1186,14,039,0001,Guadalajara,POLYGON ((-103.3135014151582 20.74374155402955...,Jalisco,Guadalajara,14039
48,02,002,0001,Mexicali,POLYGON ((-115.4003165851423 32.67198258432887...,Baja California,Mexicali,02002
2280,19,039,0001,Monterrey,POLYGON ((-100.4206058097103 25.74422449616361...,Nuevo León,Monterrey,19039
1943,16,053,0001,Morelia,POLYGON ((-101.2098173996532 19.75286704607737...,Michoacán de Ocampo,Morelia,16053
